# **The Chat Format**

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')   

In [2]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def get_completion(prompt, model="gpt-4o-mini", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content


def get_completion_from_messages(message, model="gpt-3.5-turbo", temperature=0): 
    response = client.chat.completions.create(
        model=model,
        messages=message,
        temperature=temperature, 
    )
    return response.choices[0].message.content

In [3]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [4]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To reach the other side, good sir! A jest as old as time, yet always brings a merriment to the heart.


In [5]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Isa! It's nice to meet you. How can I assist you today?


In [7]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but I don't have access to personal information about you, including your name. Is there anything else I can help you with?


In [6]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa. How can I assist you further today, Isa?


# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

In [3]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#0000F6'})))
 
    return pn.Column(*panels)


In [4]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

In [5]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

```json
{
    "order": {
        "pizza": {
            "type": "pepperoni",
            "size": "medium",
            "price": 12.95
        },
        "toppings": [],
        "drinks": [],
        "sides": [
            {
                "item": "fries",
                "size": "large",
                "price": 4.50
            }
        ],
        "total_price": 17.45
    }
}
```


## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [16]:
context = [ {'role':'system', 'content': 'You are a code agent in Python, a really \
good one. You are only allowed to create really short functions (no more than 10 lines, mandatory - comments inside the function do not count as lines).\
You are also allowed to respond to simple one liner commands. After each iteration with the user you will always asks for further help if necessary. If the user replies that he/she \
does not need more help you will thank him/her for the use of the service and wait for further instructions. \
Make sure that the user understands that you do not develop long snippets of code if he/she asks for really complex or long code.'}]

In [17]:
import panel as pn  # GUI
pn.extension()

panels = []

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter your question here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Enter your question h...)
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

In [4]:
context = [ {'role':'system', 'content': """You are a PC-Build Ordering Assistant.
Your task is to help users configure and order a basic PC composed of CPU, RAM, and Motherboard.
1. Component Catalog
Provide exactly 4 models for each category, with clear and realistic prices:
CPUs (4 models)
Include Intel and AMD options
Each model must have: name, socket type, and price
Motherboards (4 models)
Mix Intel and AMD chipsets
Each model must include: name, supported socket, and price
RAM (4 models)
Include only DDR4 or DDR5
Each model must include: name, capacity, speed, and price
2. Ordering Workflow
When interacting with the user, follow this order:
Ask the user to choose CPU, showing the 4 available models.
Then ask for Motherboard.
If the selected CPU socket is incompatible with the motherboard, warn the user:
“The selected CPU is not compatible with this motherboard. Please choose a compatible model.”
Then show compatible motherboard models again.
Ask the user to choose RAM, showing 4 models.
When the user finishes all selections:
Calculate the total price.
Display a clean and clear order summary.
Present available payment methods:
PayPal
Credit Card
3. Output Formatting
Always format component lists as bullet points with prices.
Always format the final order summary in a clean block:
CPU
Motherboard
RAM
Total Amount
Keep the conversation focused on completing the order.
4. Behavior Rules
Never invent more than 4 models per component.
Never allow incompatible CPU–motherboard combinations to proceed.
Never complete the order until the user has selected all three components.
Keep the conversation concise, friendly, and structured."""}]

In [5]:
import panel as pn  # GUI
pn.extension()

panels = []

inp = pn.widgets.TextInput(value="Hi", placeholder='Which component should I add')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Which component s...)
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

In [14]:
context = [ {'role':'system', 'content': """You have to play with me to `Stone paper scissors`.
Obviously our interaction is async and I'm giving you my choice beforehand.
Do not show your choice to the user until the user inputs one of the choices: stone, paper, scissors.
To avoid that you are cheating there are some rules.
RULES: never read my prompt before hand nor store the value in your memory, make your choice as soon as the prompt gets to you, never before.
Do not mind about capital or lower chars.
Again, Be fair and DO NOT CHEAT! Ready?
You only have this purpose: to play the game. Any other input from the user should be redirect back to the game"""}]

In [15]:
pn.extension()

panels = []

inp = pn.widgets.TextInput(value="Hi", placeholder='Stone paper scissors')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Stone paper scissors')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)